In [1]:
#Install
!pip install catboost
!pip install xlsxwriter
!pip install catboost
!pip install scikit-optimize
!pip install xgboost
!pip install lightgbm
!pip install openpyxl

In [2]:
import os
import pandas as pd
from numpy import mean
from numpy import std
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import *
from threading import Thread
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.base import BaseEstimator, ClassifierMixin
from csv import writer

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
import csv
import time
import warnings

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from openpyxl import Workbook
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from sklearn.exceptions import ConvergenceWarning


In [3]:
def load_spectrum_file(dir_path, max_subfolders=None, max_files_per_subdir=None, max_rows=None):
    """Carrega arquivos de espectro e associa rótulos numéricos às subpastas."""
    data, labels = [], []
    subfolder_labels = {}
    label_encoder = LabelEncoder()

    # Contadores para controlar a quantidade de subpastas e arquivos lidos
    subfolder_count = 0

    for root, dirs, files in os.walk(dir_path):
        # Limitar a quantidade de subpastas
        if max_subfolders is not None and subfolder_count >= max_subfolders:
            break

        for dir_name in dirs:
            subfolder_path = os.path.join(root, dir_name)
            subfolder_labels[subfolder_path] = dir_name
            subfolder_count += 1

            # Limitar a quantidade de subpastas
            if max_subfolders is not None and subfolder_count > max_subfolders:
                break

            file_count = 0
            for filename in os.listdir(subfolder_path):
                if filename.endswith('.csv'):
                    if max_files_per_subdir is not None and file_count >= max_files_per_subdir:
                        break
                    file_path = os.path.join(subfolder_path, filename)
                    try:
                        df = pd.read_csv(file_path, delimiter=',', nrows=max_rows)
                        y = df.iloc[:, 1].values  # Pega os valores da segunda coluna

                        # Adiciona o rótulo da subpasta à lista de labels
                        labels.append(subfolder_labels[subfolder_path])
                        data.append(y)
                        file_count += 1
                    except Exception as e:
                        print(f"Error reading file {file_path}: {e}")

    # Transforma os rótulos das subpastas em números
    labels = label_encoder.fit_transform(labels)
    return np.array(data), np.array(labels), label_encoder.classes_

# Exemplo de uso
print("Loading data...")
data, labels, class_names = load_spectrum_file(
    'Data_csv/',
    max_subfolders=None,
    max_files_per_subdir=None,
    max_rows=None
)
print("                                   ")
print("-----------------------------------")
print("                                   ")
print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Class names: {class_names}")
print("                                   ")
print("-----------------------------------")
print("                                   ")

X_=data
#print(X_)
y=labels
#X_ = np.nan_to_num(data)  # Substitui NaN e inf por zero

#print(X_)

Loading data...
                                   
-----------------------------------
                                   
Data shape: (5292, 1570)
Labels shape: (5292,)
Class names: ['Actinolite' 'Adamite' 'Aegirine' 'Aenigmatite' 'Akermanite' 'Albite'
 'Almandine' 'Amblygonite' 'Amesite' 'Analcime' 'Anatase' 'Andalusite'
 'Andradite' 'Anglesite' 'Anhydrite' 'Ankerite' 'Annite' 'Anorthite'
 'Aragonite' 'Arfvedsonite' 'Arsenopyrite' 'Astrophyllite' 'Atacamite'
 'Augelite' 'Augite' 'AxiniteFe' 'Azurite' 'Baddeleyite' 'Barrerite'
 'Barysilite' 'Baryte' 'Bertrandite' 'Beryl' 'Beudantite' 'Bikitaite'
 'Bobdownsite' 'Braunite' 'Brazilianite' 'Brookite' 'Brucite' 'Bustamite'
 'Calcite' 'Cassiterite' 'Catapleiite' 'Celestine' 'Cerussite'
 'ChabaziteCa' 'Chloritoid' 'Chondrodite' 'Chrysoberyl' 'Clinochlore'
 'Clinohumite' 'ClinoptiloliteCa' 'Clinozoisite' 'ColumbiteFe'
 'Conichalcite' 'Cordierite' 'Corundum' 'Covellite' 'Creedite' 'Crocoite'
 'Cuprite' 'Danburite' 'Datolite' 'Diamond' 'Diaspo

In [4]:
# Definição do experimento
def experiment(model_name, model, params, X_train, X_test, y_train, y_test, repet):
    
    # Caminho da pasta para salvar o arquivo (cria uma pasta para cada modelo)
    model_dir = os.path.join('Results/', model_name)

    # Verifica se o diretório existe, se não, cria o diretório
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    # Nome do arquivo para salvar os resultados
    results_file = os.path.join(model_dir, f'{model_name}_repetição_{repet}.csv')
    
    # configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
    
    # define search
    search = BayesSearchCV(model, params, scoring='accuracy', cv=cv_inner, n_iter=3, refit=True, random_state=1, n_jobs=1)

    # execute search
    result = search.fit(X_train, y_train)

    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)

    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    prec = precision_score(y_test, yhat, average='weighted', zero_division=1)
    rec = recall_score(y_test, yhat, average='weighted')
    f1 = f1_score(y_test, yhat, average='weighted')
    mcc = matthews_corrcoef(y_test, yhat)

    # store the result
    results = [[model_name, acc, rec, prec, f1, result.best_score_, result.best_params_]]

    # report progress
    print(f"Repetição {repet}: {model_name} > acc={acc:.3f}, est={result.best_score_:.3f}, cfg={result.best_params_}")

    # save results to CSV, each repetition in a new file
    df = pd.DataFrame(results, columns=['model', 'acc', 'rec', 'prec', 'f1', 'best_score', 'best_params'])
    df.to_csv(results_file, index=False)

In [5]:
model_params = {
           
    #00 Logistic Regression
          'lr': {'model': LogisticRegression(),
                'params': {
                            'C': Real(1e-4, 1e4, prior='log-uniform'),
                            'fit_intercept': Categorical([True, False]),
                            'solver': Categorical(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
                            'max_iter': [500],
                            'random_state': [1],
                            'n_jobs': [1]  # Limita para 1 thread
                         }},
            
    #01 K-Nearest Neighbors
          'knn': {'model': KNeighborsClassifier(),
                  'params': {
                            'n_neighbors': Integer(1, 50),
                            'weights': Categorical(['uniform', 'distance']),
                            'algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
                            'p': Integer(1, 5),
                            'n_jobs': [1]  # Limita para 1 thread
                         }},
    
    #02 Gaussian Naive Bayes
          'nb': {'model': GaussianNB(),
                'params': {
                         'var_smoothing': Real(1e-10, 1e-1, prior='log-uniform')}},
    
    #03 Decision Tree
          'dt': {'model': DecisionTreeClassifier(),
                'params': {
                            'criterion': Categorical(['gini', 'entropy']),
                            'splitter': Categorical(['best', 'random']),
                            'max_depth': Integer(3, 30),
                            'min_samples_split': Integer(2, 10),
                            'min_samples_leaf': Integer(1, 10),
                            'max_features': Real(0.1, 1.0, prior='uniform'),
                            'random_state': [1]}},

    #04 Support Vector Machine
          'svm': {'model': SVC(),
                  'params': {
                            'C': Real(2**-5, 2**5, prior='log-uniform'),
                            'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
                            'degree': Integer(2, 5),  # Relevante para o kernel 'poly'
                            'coef0': Real(0, 1),      # Relevante para os kernels 'poly' e 'sigmoid'
                            'gamma': Real(2**-9, 2**1, prior='log-uniform'),
                            'random_state': [1]}},
            
    #05 Gaussian Process Classifier
          #'gpc': {'model': GaussianProcessClassifier(),
          #        'params': {
          #                  'optimizer': Categorical(['fmin_l_bfgs_b', None]),
          #                  'n_restarts_optimizer': Integer(0, 10),
          #                  'max_iter_predict': [500],
          #                  'random_state': [1]}},
            
    #06 Multi-Layer Perceptron
          #'mlp': {'model': MLPClassifier(),
          #        'params': {
          #                  'hidden_layer_sizes': Integer(10,100),
          #                  'activation': Categorical(['logistic', 'tanh', 'relu']),
          #                  'solver': Categorical(['sgd', 'adam']),
          #                  'max_iter': [5000],
          #                  'random_state': [1]}},
            
    #07 Ridge Classifier
          'ridge': {'model': RidgeClassifier(),
                    'params': {
                                'alpha': Real(1e-4, 1e4, prior='log-uniform'),
                                'fit_intercept': Categorical([True, False]),
                                'solver': Categorical(['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']),
                                'random_state': [1],
                                'n_jobs': [1]  # Limita para 1 thread
                              }},
            
    #08 Random Forest
          'rf': {'model': RandomForestClassifier(),
                'params': {
                          'n_estimators': Integer(10, 500),
                          'criterion': Categorical(['gini', 'entropy']),
                          'max_depth': Integer(3, 30),
                          'random_state': [1],
                          'n_jobs': [1]  # Limita para 1 thread
                         }},
            
    #09 Quadratic Discriminant Analysis
          #'qda': {'model': QuadraticDiscriminantAnalysis(),
          #        'params': {
          #                  'reg_param': Real(0, 1, prior='uniform'),
          #                  'store_covariance': Categorical([True, False]),
          #                  'tol': Real(1e-5, 1e-1, prior='log-uniform')}},
            
    #10 AdaBoost
          'ada': {'model': AdaBoostClassifier(),
                  'params': {
                            'n_estimators': Integer(10, 500),
                            'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                            'algorithm': Categorical(['SAMME']),
                            'random_state': [1]}},
            
    #11 Linear Discriminant Analysis
         #'lda': {'model': LinearDiscriminantAnalysis(),
         #        'params': {
         #               'solver': Categorical(['lsqr', 'eigen', 'auto']),
         #               'shrinkage': Real(0, 1, prior='uniform'),
         #               'tol': Real(1e-6, 1e-4, prior='log-uniform')}},
            
    #12 Extra Trees Classifier
          'et': {'model': ExtraTreesClassifier(),
                 'params': {
                         'n_estimators': Integer(10, 500),
                         'criterion': Categorical(['gini', 'entropy']),
                         'max_depth': Integer(3, 30),
                         'n_jobs': [1]  # Limita para 1 thread
                       }},
            
    #13 XGBoost
          'xgboost': {'model': XGBClassifier(),
                      'params': {
                                'learning_rate': Real(0.01, 0.3, prior='uniform'),
                                'n_estimators': Integer(50, 500),
                                'max_depth': Integer(3, 10),
                                'gamma': Real(0, 1, prior='uniform'),
                                'n_jobs': [1]  # Limita para 1 thread
                              }},
           
    #14 LightGBM
          'lightgbm': {'model': LGBMClassifier(verbose=-1),
                      'params': {
                                'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                                'n_estimators': Integer(10, 500),
                                'num_leaves': Integer(2, 100),
                                'max_depth': Integer(3, 10),
                                'n_jobs': [1]  # Limita para 1 thread
                               }},
           
    #15 CatBoost
          'catboost': {'model': CatBoostClassifier(verbose=0),
                     'params': {
                               'learning_rate': Real(1e-3, 1, prior='log-uniform'),
                               'iterations': Integer(10, 500),
                               'depth': Integer(3, 10),
                               'l2_leaf_reg': Real(1, 10, prior='uniform'),
                               'border_count': Integer(1, 255),
                               'bagging_temperature': Real(0, 1, prior='uniform'),
                               'random_strength': Real(1e-9, 10, prior='log-uniform'),
                               'thread_count': [1]  # Limita para 1 thread
                             }}
}


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Lista para armazenar a referência dos processos
process = []

# Cria o pool de threads
pool = ProcessPoolExecutor(max_workers=32)

for model_name, mp in model_params.items():
#for model_name, mp in [list(model_params.items())[9]]:
    
    # Loop para realizar 30 iterações de train_test_split e executar experimentos
    for i in range(30):
        
        # Realiza o particionamento dos dados
        X_train, X_test, y_train, y_test = train_test_split(X_, y, train_size=0.7, test_size=0.3, random_state=i, shuffle=True, stratify=y)
        
        # Adiciona o experimento na lista de threads
        exp = pool.submit(experiment, model_name, mp['model'], mp['params'], X_train, X_test, y_train, y_test, i + 1)

        # Armazena a referência da thread
        process.append(exp)
        
        #time.sleep(0.01)
            
        #print(i,model_name)
    

# Aguarda a finalização das process
#for exp in as_completed(process):
#    try:
#        exp.result()
#    except Exception as e:
#        print(f"Exception occurred: {e}")

# Fecha o pool
pool.shutdown(wait=True)

Repetição 2: knn > acc=0.931, est=0.897, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 1: knn > acc=0.936, est=0.902, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 4: knn > acc=0.931, est=0.901, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 3: knn > acc=0.936, est=0.896, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 5: knn > acc=0.930, est=0.895, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 6: knn > acc=0.933, est=0.893, cfg=OrderedDict([('algorithm', 'kd_tree'), ('n_jobs', 1), ('n_neighbors', 2), ('p', 3), ('weights', 'uniform')])
Repetição 7: knn > acc=0.935, est=0.902, cfg=OrderedDict([('algorithm', 'kd_